# import dependencies

In [18]:
import os
import sys
import pandas as pd
import re
import pickle

# Obtener la ruta del directorio padre
parent_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Añadir la ruta del directorio padre al path de Python
sys.path.append(parent_path)
from connect import *
from dataStructure import *

# Paths to raw data

In [19]:
raw_data_folder_path = os.path.join(parent_path,'rawData')

# Loading and cleaning data

In [20]:
raw_data_name_files = os.listdir(raw_data_folder_path)

In [138]:
file_to_read = 'TES3Z'
file = [name_file for name_file in raw_data_name_files if file_to_read in name_file][0]
path_file = os.path.join(raw_data_folder_path,file)
df = pd.read_excel(path_file, index_col=0)

In [139]:
df

,3Z2_74_21,3Z2_53_13,3Z2_52_12,3Z4_64_32,3Z4_53_13,3Z4_73_33,3Z4_42_21,3Z6_73_33,3Z6_54_MoAu,3Z6_42_21,...,3Z2_53_22,3Z2_54_34,3Z2_74_41,3Z2_64_32,3Z2_53_13.1,3Z2_54_24,3Z2_53_22.1,3Z2_63_22,3Z2_64_33,3Z2_44_24
Membrana,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
stackThickMoAuAu (nm),35/15/180,35/15/180,35/15/180,45/15/250,45/15/250,45/15/250,45/15/250,35/15/160,35/15/160,35/15/160,...,35/15/180,35/15/180,35/15/180,35/15/180,35/15/180,35/15/180,35/15/180,35/15/180,35/15/180,35/15/180
stack Area WxL (um),16 x 16,16 x 10,16 x 24,40 x 16,16 X 10,80 X 120,10 X 120,80 X 120,No,10 X 120,...,30 x 10,80 x 16,80 x 80,40 x 16,16 x 10,20 x 16,30 x 10,12 x 40,60 x16,40 x 80
Absorbente,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
Tags,EB,EB,EB,EB,EB,EB,EB,EB,ET,EB,...,EB,EB,EB,EB,EB,EB,EB,EB,EB,EB
RN Ohms,27,16,47,12,18,43,302,80,1.28,580,...,10,5,35,11,16,20,15,78,9,55
Tc0 mK,140,185,110,190,193,209,196,136,113,108,...,123,107,104,89,100,79,95,81,96,85
Tags_,Dañado,Funcional,Funcional,Funcional,Funcional,Funcional,Funcional,Funcional,Funcional,Funcional,...,Funcional,Funcional,Funcional,Funcional,Funcional,Funcional,Dañado,Funcional,Funcional,Funcional
Medida,A,A,A,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,No info,N,N
Holder,A,A,A,N,N,N,N,N,N,N,...,A,A,N,N,N,N,A,No info,N,N


In [140]:
tes = {
    'tesName': [],
    'tags':[],
    'stackw':[],
    'stackl':[],
    'stackMoT':[],
    'stackAu1T':[],
    'stackAu2T':[],
    'absorber':[],
    'membrane':[],
    'TC0':[],
    'RN':[],
    'cryo':[],
    'holder':[],
    'caracteristicas':[]
}

def verifica_patron_NZN(cadena):
    '''
    patron a verificar número Z número
    '''
    patron = r"\d[Z]\d+$"
    return bool(re.match(patron, cadena))

def verifica_patron_2d(cadena):
    '''
    Verifica dos dígitos
    '''
    patron = r"^\d{2}$"
    return bool(re.match(patron, cadena))

def verifica_patron_3numeros(cadena):
    '''
    Verifica que la cadena de texto contenga como máximo tres números
    '''
    patron = '^\D*(\d\D*){1,3}$'
    return bool(re.match(patron, cadena))



def clean_names(name:str, i:int)->str:
    '''
    clean strings names
    '''
    name_list = name.split('_')

    if len(name_list) != 3:
        print(f'The name {name} needs further inspection, position {i}')
        final_name = name
        return final_name

    if verifica_patron_NZN(name_list[0]) == True:
        final_name =name_list[0]+'_'
    else:
        print(f'The name {name} needs further inspection, position {i}')
        final_name = name
        return final_name
    
    if verifica_patron_2d(name_list[1].strip()) == True:
        final_name = final_name+name_list[1].strip()+'_'
    else:
        print(f'The name {name} needs further inspection, position {i}')
        final_name = name
        return final_name
    
    if verifica_patron_2d(name_list[2].strip()[:2]) == True or i == 8:
        if verifica_patron_2d(name_list[2].strip()[:2]) == True:
            final_name = final_name+name_list[2].strip()[:2]
        else:
            final_name = final_name+name_list[2].strip()
    else:
        print(f'The name {name} needs further inspection, position {i}')
        final_name = name
        return final_name
    
    return final_name


def clean_width_length(raw:str, i:int)->List:

    data = raw.lower().split('x')

    try:
        data = [int(i.strip()) for i in data]
    except Exception as error:
        data = [-1,-1]
        message = f"An error occurred: {error} and the data {i} was not properly transformed"
        print(message)
    
    return data

def clean_Mo_Au_Au(raw:str, i:int)->List:
    final_data = []
    data = raw.lower().split('/')

    if len(data)!=3:
        final_data  = data
        print(f'Verifica dato en posición {i}')
        return final_data
        
    for d in data:
        if verifica_patron_3numeros(d.strip()) == True:
            final_data.append(int(d.strip()))
        else:
            print(f'Verifica dato en posición {i}')
            return final_data
    return final_data

def select_cryo(raw:str, i:int)->str:
    
    var = raw.strip().lower()

    if var == 'a':
        cryo_name = 'Kelvinox'
        return cryo_name
    elif var == 'n':
        cryo_name ='BlueFors'
        return cryo_name
    else:
        cryo_name = 'no info'
        return cryo_name

def select_holder(raw:str, i:int)->str:
    
    var = raw.strip().lower()

    if var == 'a':
        holder = 'antiguo'
        return holder
    elif var == 'n':
        holder ='nuevo'
        return holder
    else:
        holder = 'no info'
        return holder

def extrae_parametros(cadena:str, i:int)->List:
    try:
        patron1 = r'\[\[(.*?)\]\]'
        patron2 = r'\[(.*?)\]'
        patron3 = r'\((.*?)\)'

        elementos = re.findall(patron1, cadena)
        elementos = '['+elementos[0]+']'
        elementos = re.findall(patron2, elementos)
        elementos = [re.findall(patron3, i) for i in elementos]
        elementos = [[list(map(str.strip,elemento.split(','))) for elemento in grupo] for grupo in elementos]
        return elementos
    except Exception as error:
        message = f"An error occurred: {error} and the data {i} was not properly transformed"
        print(message)
        return 'error'
    
tes['tesName'] = [clean_names(name, i) for i,name in enumerate(df.columns)]

tes['tags'] = list(zip(*[[row[name] for name in df.columns] for n, row in df.iterrows() if n == 'Tags' or n == 'Tags_']))

wl = list(zip(*[[clean_width_length(data,i) for i, data in enumerate(row)] for n, row in df.iterrows() if n == 'stack Area WxL (um)'][0]))

tes['stackw'] = list(wl[0])

tes['stackl'] = list(wl[1])

mo1o2 = list(zip(*[[clean_Mo_Au_Au(data,i) for i, data in enumerate(row)] for n, row in df.iterrows() if n == 'stackThickMoAuAu (nm)'][0]))

tes['stackMoT'] = list(mo1o2[0])

tes['stackAu1T'] = list(mo1o2[1])

tes['stackAu2T'] = list(mo1o2[2])

tes['absorber'] = [[data.lower().strip() for data in row] for n, row in df.iterrows() if n == 'Absorbente'][0]

tes['membrane'] = [[data.lower().strip() for data in row] for n, row in df.iterrows() if n == 'Membrana'][0]

tes['TC0'] = [[int(data) for data in row] for n, row in df.iterrows() if n == 'Tc0 mK'][0]

tes['RN'] = [[float(data) for data in row] for n, row in df.iterrows() if n == 'RN Ohms'][0]

tes['cryo'] = [[select_cryo(data,i) for i, data in enumerate(row)] for n, row in df.iterrows() if n == 'Medida'][0]

tes['holder'] = [[select_holder(data,i) for i, data in enumerate(row)] for n, row in df.iterrows() if n == 'Holder'][0]

tes['caracteristicas'] = [[extrae_parametros(data,i) for i, data in enumerate(row)] for n, row in df.iterrows() if n == 'datos'][0]

df_raw = pd.DataFrame.from_dict(tes)

df_raw.head(10)


An error occurred: invalid literal for int() with base 10: 'no' and the data 8 was not properly transformed


,tesName,tags,stackw,stackl,stackMoT,stackAu1T,stackAu2T,absorber,membrane,TC0,RN,cryo,holder,caracteristicas
0,3Z2_74_21,"(EB, Dañado)",16,16,35,15,180,no,no,140,27.00,Kelvinox,antiguo,"[[[3 uA, normal, redondeo, reversible], [10 uA..."
1,3Z2_53_13,"(EB, Funcional)",16,10,35,15,180,no,no,185,16.00,Kelvinox,antiguo,"[[[3 uA, normal, redondeo, reversible], [10 uA..."
2,3Z2_52_12,"(EB, Funcional)",16,24,35,15,180,no,no,110,47.00,Kelvinox,antiguo,"[[[3 uA, normal, irreversible]]]"
3,3Z4_64_32,"(EB, Funcional)",40,16,45,15,250,no,no,190,12.00,BlueFors,nuevo,"[[[3 uA, normal, reversible]]]"
4,3Z4_53_13,"(EB, Funcional)",16,10,45,15,250,no,no,193,18.00,BlueFors,nuevo,"[[[3 uA, normal, redondeo reversible]]]"
5,3Z4_73_33,"(EB, Funcional)",80,120,45,15,250,no,no,209,43.00,BlueFors,nuevo,"[[[3 uA, cola, irreversible]]]"
6,3Z4_42_21,"(EB, Funcional)",10,120,45,15,250,no,no,196,302.00,BlueFors,nuevo,"[[[3 uA, normal, reversible], [10 uA, doble tr..."
7,3Z6_73_33,"(EB, Funcional)",80,120,35,15,160,no,no,136,80.00,BlueFors,nuevo,"[[[3 uA, cola, reversible]]]"
8,3Z6_54_MoAu,"(ET, Funcional)",-1,-1,35,15,160,no,no,113,1.28,BlueFors,nuevo,"[[[3 uA, cola, reversible]]]"
9,3Z6_42_21,"(EB, Funcional)",10,120,35,15,160,no,no,108,580.00,BlueFors,nuevo,"[[[3 uA, normal, irreversible]]]"


In [141]:
df_raw[df_raw['tesName'].duplicated()]

,tesName,tags,stackw,stackl,stackMoT,stackAu1T,stackAu2T,absorber,membrane,TC0,RN,cryo,holder,caracteristicas
36,3Z2_74_21,"(EB, Dañado)",16,16,35,15,180,no,no,105,27.0,Kelvinox,antiguo,"[[[3 uA, normal, redondeo, reversible], [10 uA..."
47,3Z2_53_13,"(EB, Funcional)",16,10,35,15,180,no,no,100,16.0,BlueFors,nuevo,"[[[3 uA, cola, reversible], [10 uA, normal, re..."
49,3Z2_53_22,"(EB, Dañado)",30,10,35,15,180,no,no,95,15.0,BlueFors,antiguo,"[[[3 uA, cola, reversible], [10 uA, normal, re..."


## Further cleaning

In [142]:
duplicated_names = df_raw['tesName'][df_raw['tesName'].duplicated()].to_list()
for d_name in duplicated_names:
    indice = []
    state = []
    for n,row in df_raw[df_raw['tesName']==d_name].iterrows():
        state.append(row['tags'][1])
        indice.append(n)
    if all(x == state[0] for x in state)==False:
        to_delete = state.index('Funcional')
        df_raw = df_raw.drop(indice[to_delete])

In [143]:
df_raw[df_raw['tesName'].duplicated()]

,tesName,tags,stackw,stackl,stackMoT,stackAu1T,stackAu2T,absorber,membrane,TC0,RN,cryo,holder,caracteristicas
36,3Z2_74_21,"(EB, Dañado)",16,16,35,15,180,no,no,105,27.0,Kelvinox,antiguo,"[[[3 uA, normal, redondeo, reversible], [10 uA..."
47,3Z2_53_13,"(EB, Funcional)",16,10,35,15,180,no,no,100,16.0,BlueFors,nuevo,"[[[3 uA, cola, reversible], [10 uA, normal, re..."


In [144]:
# drop raw of a pandas dataframe by index 47
df_raw = df_raw.drop(36)
df_raw.reset_index(drop=True, inplace=True)

In [145]:
df_raw[df_raw['tesName']== '3Z2_53_13']

,tesName,tags,stackw,stackl,stackMoT,stackAu1T,stackAu2T,absorber,membrane,TC0,RN,cryo,holder,caracteristicas
1,3Z2_53_13,"(EB, Funcional)",16,10,35,15,180,no,no,185,16.0,Kelvinox,antiguo,"[[[3 uA, normal, redondeo, reversible], [10 uA..."
45,3Z2_53_13,"(EB, Funcional)",16,10,35,15,180,no,no,100,16.0,BlueFors,nuevo,"[[[3 uA, cola, reversible], [10 uA, normal, re..."


# Saving data

In [146]:
store_path = os.path.join(parent_path,'OutputData')
file_path_out = os.path.join(store_path,'df_raw.pickle')
with open(file_path_out, 'wb') as handle:
    pickle.dump(df_raw,handle)

# Bulk data

In [147]:
path_to_bulk_data = os.path.join(parent_path,'rawData','datos_2023_01_15')
files = os.listdir(path_to_bulk_data)

In [148]:
def construir_dataframes(file,df):
    try:
        if len(df.columns) < 2:
            raise ValueError("El DataFrame debe tener al menos dos columnas")
        elif len(df.columns) % 2 != 0:
            raise ValueError("El número de columnas en el DataFrame debe ser par")
        else:
            output_data = {
                "tesName": [],
                "dataframe_fileName": [],
                "current": [],
                "dataframe": []
            }
            tesName = '_'.join(file.split(".")[0].strip().split(" ")[1].split("_")[0:3])
            for i in range(0, len(df.columns), 2):
                df_temporal = df.iloc[:, i:i+2].dropna(how="all")
                nombre_columnas = df_temporal.columns
                corriente = [float(re.findall(r'\d+', col)[0]) for col in nombre_columnas if "r" in col.lower().strip()][0]
                output_data["tesName"].append(tesName)
                output_data["dataframe_fileName"].append(file)
                output_data["current"].append(corriente)        
                output_data["dataframe"].append(df.iloc[:, i:i+2].dropna(how="all"))
            return output_data
    except Exception as error:
        message = f"An error occurred: {error} and the data {file} was not properly transformed"
        print(message)
        return []

In [149]:
list_data = [construir_dataframes(file, pd.read_excel(os.path.join(path_to_bulk_data,file))) for file in files if file[-3:]=='xls']

In [150]:
dict_data = {}
for data in list_data:
    dict_data[data['tesName'][0]] = data

In [151]:
store_path = os.path.join(parent_path,'OutputData')
file_path_out = os.path.join(store_path,'dict_data_from_files.pickle')
with open(file_path_out, 'wb') as handle:
    pickle.dump(dict_data,handle)